In [1]:
import plotly.express as px

In [13]:
# Load the labels
def load_labels(filename: str) -> dict:
    
    dict_storage = {}
    
    with open(filename) as file:
        for index, line in enumerate(file):
            dict_storage[index] = line.strip().split(' ')
    
    return dict_storage

dialog_act_dict = load_labels('dailydialog/dialogues_act.txt')
emotion_dict = load_labels('dailydialog/dialogues_emotion.txt')

In [14]:
# Validation

for index, act_labels in dialog_act_dict.items():
    
    emotion_labels = emotion_dict[index]
    
    if len(emotion_labels) != len(act_labels):
        print(index)

In [ ]:
# Mappers

def dialog_act_mapping() -> dict:
    
    return {
        '1': 'inform', '2': 'question', 
        '3': 'directive', '4': 'commissive'
    }

def emotion_mapping() -> dict:
    
    return {
        '0': 'neutral', '1': 'anger', '2': 'disgust', 
        '3: fear, 4: happiness, 5: sadness, 6: surprise}
    }

# DailyDialog Dataset Analysis

This analysis is divided into the following parts:

- Dialog Act and Emotion labels
- Utterance-related
- Topic-related

# Dialog Act and Emotion labels

The following sections looks at the visualizations, in relation to dialog act and emotion labels

The visualizations are:
